In [3]:
import pandas as pd
import requests
import re

from BC_solutions import Databases
from datetime import datetime

# POD data

In [6]:
# Database credentials
server = 'CorpPolDB'
database = 'DW'
UID = 'POD'
PWD = 'POD2019'

database_manager = Databases.DatabaseManager(server, database, UID, PWD)

POD POD2019
Connection with database established.


In [7]:
df_pod = database_manager.get_data('SELECT * FROM [DW].[pod].[v_POD]')
df_pod.to_csv('POD_raw.csv', index=False)

# Geographic data

In [5]:
df_farm_gps = pd.read_excel("FarmGPS.xlsx")

In [6]:
df_farm_gps

,FarmID,Complete GPS,Latitude,Longitude
0,AG10,NaN,6.838791,-3.497348
1,AG11,NaN,6.647595,-3.353233
2,AG130,NaN,6.642344,-3.960821
3,AG201,NaN,6.717100,-3.595337
4,AG303,NaN,6.479306,-3.930839
...,...,...,...,...
153,SP18,N5° 55.320' W6° 20.918',NaN,NaN
154,SP25,N5° 13.226' W6° 49.270',NaN,NaN
155,SP30,N5° 25.934' W6° 37.572',NaN,NaN
156,SP31,N5°27.158' W6°41.377',NaN,NaN


In [7]:
def convert_gps_to_lat_long(gps):
    """Function responsible for converting specific type of GPS coordinates to the latitude and longitude format.
    REMEMBER: In order for the function to work properly, the input string must not consist of any whitespaces.

    Parameters
    ----------
    gps : str
        GPS to convert.

    Returns
    -------
    float, float
        Latitude and longitude extracted from GPS.
    """
    
    pattern = r'([NS])(\d+)°([\d.]+)\'([WE])(\d+)°([\d.]+)\''
    match = re.match(pattern, gps)

    lat_dir = match.group(1)
    lat_deg = int(match.group(2))
    lat_min = float(match.group(3))
    lon_dir = match.group(4)
    lon_deg = int(match.group(5))
    lon_min = float(match.group(6))

    latitude = lat_deg + lat_min / 60.0
    longitude = lon_deg + lon_min / 60.0

    if lat_dir == 'S':
        latitude = -latitude
    if lon_dir == 'W':
        longitude = -longitude

    return latitude, longitude

In [8]:
df_farm_gps.loc[df_farm_gps["Latitude"].isnull(), "Complete GPS"] = df_farm_gps[df_farm_gps["Latitude"].isnull()]["Complete GPS"].str.replace(" ", "")
for index, row in pd.DataFrame(df_farm_gps.loc[df_farm_gps["Latitude"].isnull(), "Latitude"]).iterrows():
    df_farm_gps.iloc[index, 2] = convert_gps_to_lat_long(df_farm_gps.iloc[index, 1])[0]
    df_farm_gps.iloc[index, 3] = convert_gps_to_lat_long(df_farm_gps.iloc[index, 1])[1]
df_farm_gps.drop(columns=["Complete GPS"], inplace=True)

In [9]:
df_farm_gps

,FarmID,Latitude,Longitude
0,AG10,6.838791,-3.497348
1,AG11,6.647595,-3.353233
2,AG130,6.642344,-3.960821
3,AG201,6.717100,-3.595337
4,AG303,6.479306,-3.930839
...,...,...,...
153,SP18,5.922000,-6.348633
154,SP25,5.220433,-6.821167
155,SP30,5.432233,-6.626200
156,SP31,5.452633,-6.689617


In [10]:
df_farm_gps.to_csv("tbl_FarmsLocation.csv", index=False)

# Weather data

In [ ]:
df_tbl_weather_daily = pd.DataFrame(columns=["Date", "FarmID", "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", 
                                              "precipitation_sum", "rain_sum", "precipitation_hours", "windspeed_10m_max", "windgusts_10m_max", "shortwave_radiation_sum",
                                              "et0_fao_evapotranspiration"])
df_tbl_weather_hourly = pd.DataFrame(columns=["Date", "FarmID", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm", 
                                              "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", "soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"])

In [ ]:
for farmid in df_farm_gps[df_farm_gps['Latitude'].isna() == False]["FarmID"]:
    latitude, longitude = df_farm_gps[df_farm_gps["FarmID"] == farmid]["Latitude"].values[0], df_farm_gps[df_farm_gps["FarmID"] == farmid]["Longitude"].values[0]
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date=1998-01-01&end_date={datetime.today().strftime('%Y-%m-%d')}&daily=temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration&hourly=soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm&timezone=GMT"
    data = requests.get(url, verify=False)
    json_data= data.json()
    df_daily, df_hourly = pd.DataFrame.from_records(json_data['daily']), pd.DataFrame.from_records(json_data['hourly'])
    df_daily["FarmID"], df_hourly["FarmID"] = farmid, farmid
    df_daily.rename(columns={'time':'Date'}, inplace=True)
    df_hourly.rename(columns={'time':'Date'}, inplace=True)
    df_daily.dropna(inplace=True)
    df_hourly.dropna(inplace=True)
    df_tbl_weather_daily = pd.concat([df_tbl_weather_daily, df_daily], ignore_index=True)
    df_tbl_weather_hourly = pd.concat([df_tbl_weather_hourly, df_hourly], ignore_index=True)

In [ ]:
df_tbl_weather_daily.to_csv("tbl_weather_daily.csv", index=False)
df_tbl_weather_hourly.to_csv("tbl_weather_hourly.csv", index=False)

In [20]:
# url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date=1998-01-01&end_date={datetime.today().strftime('%Y-%m-%d')}&daily=temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration&hourly=soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm&timezone=GMT"
# url = "https://archive-api.open-meteo.com/v1/archive?latitude=6.866667&longitude=-6.466667&elevation=276&start_date=2020-01-01&end_date=2023-05-01&daily=rain_sum&timezone=GMT"
url = f"https://archive-api.open-meteo.com/v1/archive?latitude=6.866667&longitude=-6.466667&start_date=2020-01-01&end_date=2023-05-01&daily=temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration&hourly=soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm&timezone=GMT"
data = requests.get(url, verify=False)

C:\Users\BC127735\AppData\Roaming\Python\Python39\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'archive-api.open-meteo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [23]:
json_data = data.json()
df_daily = pd.DataFrame.from_records(json_data['daily'])
df_hourly = pd.DataFrame.from_records(json_data['hourly'])

In [24]:
df_daily

,apparent_temperature_max,apparent_temperature_min,et0_fao_evapotranspiration,precipitation_hours,precipitation_sum,rain_sum,shortwave_radiation_sum,temperature_2m_max,temperature_2m_min,time,windgusts_10m_max,windspeed_10m_max
0,35.6,25.4,4.43,2.0,0.4,0.4,19.01,33.1,22.7,2020-01-01,20.5,11.0
1,32.9,21.8,4.98,0.0,0.0,0.0,20.34,32.7,21.2,2020-01-02,22.0,10.1
2,32.4,16.2,5.57,0.0,0.0,0.0,22.46,32.9,18.2,2020-01-03,24.1,10.2
3,32.2,14.7,5.92,0.0,0.0,0.0,22.56,32.8,17.8,2020-01-04,27.0,13.5
4,32.1,15.0,5.26,0.0,0.0,0.0,22.51,32.6,17.4,2020-01-05,21.6,11.3
...,...,...,...,...,...,...,...,...,...,...,...,...
1212,37.7,27.6,5.36,0.0,0.0,0.0,23.35,33.4,23.7,2023-04-27,32.8,12.2
1213,37.5,28.7,4.03,8.0,4.3,4.3,18.21,31.7,24.3,2023-04-28,33.8,13.4
1214,38.8,27.6,5.12,1.0,0.5,0.5,22.97,33.0,23.5,2023-04-29,27.4,11.4
1215,38.8,28.1,4.30,5.0,7.9,7.9,19.88,32.4,23.8,2023-04-30,23.8,11.2


In [25]:
df_hourly

,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,time
0,0.325,0.347,26.9,28.0,2020-01-01T00:00
1,0.325,0.347,26.4,28.0,2020-01-01T01:00
2,0.325,0.347,26.0,27.8,2020-01-01T02:00
3,0.325,0.347,25.7,27.7,2020-01-01T03:00
4,0.325,0.347,25.5,27.6,2020-01-01T04:00
...,...,...,...,...,...
29203,0.463,0.423,27.5,27.8,2023-05-01T19:00
29204,0.462,0.424,27.2,27.7,2023-05-01T20:00
29205,0.461,0.424,26.8,27.7,2023-05-01T21:00
29206,0.460,0.424,26.5,27.6,2023-05-01T22:00
